In [74]:
import pandas as pd
import numpy as np 
from tqdm.auto import tqdm
tqdm.pandas()

bho_cands = pd.read_pickle("/home/mcollardanuy/PlaceLinking/toponym_matching/ranker_results/bho_queries_britwikidata_candidates_wikigaz_en_002.pkl")
#bho_cands = bho_cands.head(100)

britwikidata_gazetteer = pd.read_pickle("/home/mcollardanuy/PlaceLinking/toponym_matching/gazetteers/britwikidata_gazetteer.pkl")
britwikidata_gazetteer.head()

,wkid,altname,source,lat,lon
0,Q63607865,Kilberry Head,english_label,55.818446,-5.667253
1,Q63607865,Kilberry,geonames,55.818446,-5.667253
2,Q63759247,Sandhurst Memorial Park,english_label,51.344000,-0.793000
3,Q63781193,Currie railway station,english_label,55.903100,-3.283500
4,Q63859929,Stannington War Memorial,english_label,53.392659,-1.548018


[('Q145', 147),
 ('Q25305', 102),
 ('Q5785', 99),
 ('Q43448', 86),
 ('Q27', 86),
 ('Q21', 78),
 ('Q9648', 67),
 ('Q18221', 61),
 ('Q23635', 61),
 ('Q35086', 59),
 ('Q25', 57),
 ('Q42314', 57),
 ('Q9676', 53),
 ('Q35794', 50),
 ('Q84', 49),
 ('Q1410', 48),
 ('Q8691', 46),
 ('Q192184', 46),
 ('Q22890', 43),
 ('Q160302', 39),
 ('Q23436', 38),
 ('Q35672', 35),
 ('Q8703', 35),
 ('Q100166', 34),
 ('Q42646', 34),
 ('Q212893', 33),
 ('Q23311', 33),
 ('Q42182', 33),
 ('Q170027', 32),
 ('Q172190', 32)]

In [75]:

def match_cands_wikidata(row,gazetteer,ranking):
    wikidata_cands = {}
    cands = row[ranking]
    for cand,score in cands.items():
        wikidataIds = gazetteer[gazetteer["altname"] == cand]["wkid"]
        for _id in wikidataIds:
            if _id not in wikidata_cands:
                wikidata_cands[_id] = score
    return wikidata_cands

tqdm.pandas()    
bho_cands['wikidata_cands'] = bho_cands.progress_apply(lambda row : match_cands_wikidata(row,britwikidata_gazetteer,"faiss_distance"), axis=1) 



In [76]:
bho_cands.head()

,query,pred_score,faiss_distance,cosine_sim,candidate_original_ids,query_original_id,num_all_searches,wikidata_cands
id,,,,,,,,
0,Griston,"{'Griston': 0.923, 'Riston': 0.9437, 'Siston':...","{'Griston': 0.0, 'Riston': 1.2324, 'Siston': 1...","{'Griston': 1.0, 'Riston': 0.9779, 'Siston': 0...","{'Griston': 514359, 'Riston': 188110, 'Siston'...",0,20,"{'Q59725806': 0.0, 'Q615956': 0.0, 'Q7336442':..."
1,Banham,"{'Banham': 0.9622, 'Banham Zoo': 0.9814, 'Been...","{'Banham': 0.0, 'Banham Zoo': 5.088, 'Beenham'...","{'Banham': 1.0, 'Banham Zoo': 0.9151, 'Beenham...","{'Banham': 37944, 'Banham Zoo': 96118, 'Beenha...",1,20,"{'Q2700979': 0.0, 'Q4855654': 5.088, 'Q2525831..."
2,Ash,"{'Ash': 0.9825, 'Esh': 0.9585, 'RAF Ash': 0.88...","{'Ash': 0.0, 'Esh': 2.518, 'RAF Ash': 3.6743, ...","{'Ash': 1.0, 'Esh': 0.9124, 'RAF Ash': 0.87, '...","{'Ash': 608253, 'Esh': 604802, 'RAF Ash': 3965...",2,20,"{'Q17542375': 0.0, 'Q4804353': 0.0, 'Q26563812..."
3,Kîlcarw,"{'Killocraw': 0.9168, 'Kilcor': 0.9261, 'Kilke...","{'Killocraw': 5.4492, 'Kilcor': 6.468, 'Kilkea...","{'Killocraw': 0.9242, 'Kilcor': 0.9098, 'Kilke...","{'Killocraw': 597570, 'Kilcor': 626486, 'Kilke...",3,20,"{'Q31093532': 5.4492, 'Q59724554': 6.468, 'Q59..."
4,Shavington,"{'Shavington': 0.974, 'Shannon Region': 0.98, ...","{'Shavington': 0.0, 'Shannon Region': 4.1762, ...","{'Shavington': 1.0, 'Shannon Region': 0.932, '...","{'Shavington': 649846, 'Shannon Region': 34489...",4,20,"{'Q23869524': 0.0, 'Q6840838': 4.1762, 'Q61191..."


In [79]:
from wikimapper import WikiMapper
from pathlib import Path
import urllib.parse

mapper = WikiMapper("/resources/wikidata2wikipedia/index_enwiki-20190420.db")

def map_wikidata2wikidump(wikidataId):
    path = "/resources/wikipedia/extractedResources/Aspects/"
    wikititles = mapper.id_to_titles(wikidataId)
    wikititles = [urllib.parse.quote(title.replace("_"," ")) for title in wikititles]
    wikidumps = [path + title+".json" for title in wikititles if Path(path + title+".json").is_file()]
    return wikidumps


single_best_match = 0
single_best_wikipedia_match = 0
single_possible_match = 0
multiple_best_match = 0
multiple_possible_match = 0
multiple_best_but_no_wiki_match = 0
no_wiki_match = 0

cutoff = 5

for index, row in bho_cands.iterrows():
    wikidata_cands = row["wikidata_cands"]
    best_match = [x for x,y in wikidata_cands.items() if y==0.0]
    if len(best_match)>0:
        if len(best_match)==1:
            single_best_match+=1
        else:
            wikipages = [[wikidataId,page] for wikidataId in best_match for page in map_wikidata2wikidump(wikidataId)]
            if len(wikipages)==1:
                single_best_wikipedia_match+=1
            elif len(wikipages)>1:
                multiple_best_match+=1
            elif len(wikipages)==0:
                multiple_best_but_no_wiki_match+=1
    else:
        selected = [x for x in wikidata_cands.keys()][:cutoff]
        wikipages = [[wikidataId,page] for wikidataId in selected for page in map_wikidata2wikidump(wikidataId)]
        if len(wikipages)==1:
            single_possible_match+=1
        elif len(wikipages)>1:
            multiple_possible_match+=1
        elif len(wikipages)==0:
            no_wiki_match+=1



print ("perfect unique match:",round(single_best_match/len(bho_cands),3))
print ("perfect unique wikipedia match:",round(single_best_wikipedia_match/len(bho_cands),3))
print ("multiple best candidates:",round(multiple_best_match/len(bho_cands),3))
print ("multiple best candidates but no wikipedia match:",round(multiple_best_but_no_wiki_match/len(bho_cands),3))
print ("single possible candidate:",round(single_possible_match/len(bho_cands),3))
print ("multiple possible candidates:",round(multiple_possible_match/len(bho_cands),3))
#print ("no single best candidate:",(len(bho_cands)-single_match-multiple_match-no_wiki_match)/len(bho_cands))
print ("no wiki page available",round(no_wiki_match/len(bho_cands),3))



perfect unique match: 0.464
perfect unique wikipedia match: 0.05
multiple best candidates: 0.12
multiple best candidates but no wikipedia match: 0.004
single possible candidate: 0.053
multiple possible candidates: 0.288
no wiki page available 0.021
